In [0]:
from pyspark.sql.utils import AnalysisException

In [0]:
catalog_name = dbutils.widgets.get("catalog_name")
schema_names = dbutils.widgets.get("database_name")
location_details = dbutils.widgets.get("location_details")

In [0]:
def assign_locations_to_schemas(catalog_name, schemas, locations):
    """
    Assign locations to schemas in order.
    Args:
        catalog_name (str): Catalog name
        schemas (list): List of 3 schema names
        locations (list): List of 3 locations (paths or external location names)
    """
    
    if len(schemas) != len(locations):
        raise ValueError(f"Schema count ({len(schemas)}) must match location count ({len(locations)})")
    
    results = []
    
    for schema, location in zip(schemas, locations):
        try:
            # Check if schema exists
            schema_exists = _schema_exists(catalog_name, schema)
            
            if schema_exists:
                # Alter existing schema
                _alter_schema_location(catalog_name, schema, location)
                results.append(f"ALTERED: {catalog_name}.{schema} -> {location}")
            else:
                # Create new schema
                _create_schema_with_location(catalog_name, schema, location)
                results.append(f"CREATED: {catalog_name}.{schema} -> {location}")
                
        except Exception as e:
            results.append(f"ERROR: {catalog_name}.{schema} -> {str(e)}")
    
    return results


In [0]:
def _schema_exists(catalog, schema):
    """Check if schema exists."""
    try:
        spark.sql(f"DESCRIBE SCHEMA `{catalog}`.`{schema}`").collect()
        return True
    except AnalysisException:
        return False

In [0]:
def _create_schema_with_location(catalog, schema, location):
    """Create schema with location."""
    location_clause = f"LOCATION `{location}`" if not location.startswith(('s3://', 'gs://', 'abfss://')) else f"LOCATION '{location}'"
    
    spark.sql(f"""
        CREATE SCHEMA IF NOT EXISTS `{catalog}`.`{schema}`
        {location_clause}
    """)

In [0]:
def _alter_schema_location(catalog, schema, location):
    """Add/update schema location."""
    location_clause = f"SET LOCATION `{location}`" if not location.startswith(('s3://', 'gs://', 'abfss://')) else f"SET LOCATION '{location}'"
    
    spark.sql(f"""
        ALTER SCHEMA `{catalog}`.`{schema}`
        {location_clause}
    """)

In [0]:
print(assign_locations_to_schemas(catalog_name, schema_names, location_details))

testing code cells below

In [0]:
%sql
create schema edap.schema_test1 
MANAGED LOCATION 'abfss://uc-csii-dna-rt-racing-dashboard@adlsedapcsiidnadevea002.dfs.core.windows.net/'